In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時的例外工具
from selenium.common.exceptions import TimeoutException

# 等待某個元素的出現
from selenium.webdriver.support.ui import WebDriverWait

# 期待元素出現並執行下一個指令
from selenium.webdriver.support import expected_conditions as EC

# 透過什麼方式選取元素
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 強制等待
from time import sleep

# 反爬蟲
import undetected_chromedriver as uc

# 執行 command 的時候用的
import os

# 整理 json 使用的工具
import json

#計算程式執行的時間
import time

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

from fake_useragent import UserAgent 
ua = UserAgent(cache= True)

my_options = uc.ChromeOptions()

my_options.add_argument("--start-maximized") # 視窗最大化
my_options.add_argument("--incognito") # 無痕
my_options.add_argument("--disable-popup-blocking") # 禁用彈出攔截
my_options.add_argument("--disable-notifiactions") # 關閉推波通知
my_options.add_argument("--lang=zh-TW") # 設定為繁體中文
my_options.add_argument("--headless")
# my_options.add_argument("--user-agent="+ua.random)

folderPath = "WanhuaRestaurant"
if os.path.exists(folderPath) != True:    
    os.makedirs(folderPath)
    


In [2]:
def page_link(link):
   
    # 自動取得Chrom driver 不用再額外取得 executable_path="chromedriver.exe"
    driver = uc.Chrome(options=my_options)
    
    # GoogleMap網址
    url = "https://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW"
    
    driver.get(url)
    
    sleep(1)
    
    # 計算每跑一次跑多久
    start = time.time()
        
    # 輸入關鍵字搜尋
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).send_keys(link)
    
    # 按下搜尋按鈕
    driver.find_element(
        By.CSS_SELECTOR,
        "button#searchbox-searchbutton"
    ).click()
        
    sleep (3)
        
    # 找搜尋元素結果的div
    focus = driver.find_elements(
        By.CSS_SELECTOR,
        "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
    )[1]

    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 5  # 最大無效滾動次數
    Flag = True
    counter = 0
        
    # 開始滾輪找到最底部
    while Flag:
            
        try:
            # 等待底部元素出現
            WebDriverWait(driver, 0.1).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "span.HlvSq") 
                )
            )
            
            break
            
        except TimeoutException:

           # offset: 拉槓到頁面頂端的距離
            offset = driver.execute_script('return arguments[0].scrollTop', focus)

            # 執行js指令捲動頁面
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)
            
            # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
            innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
            
            # 如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表搜尋可能逾時
            if offset == innerHeight:
                count += 1                
              
            # 計數器等於限制數則搜尋確實逾時了,需重新整理並搜尋
            if count == limit:
                
                counter+=1
                
                print(f"{link}需重新整理,搜尋逾時,本次刷新第{counter}次")
                
                # 網頁重新整理
                driver.execute_script('window.location.replace(window.location.href)')
                
                # 重新搜尋
                driver.find_element(
                    By.CSS_SELECTOR,
                    "button#searchbox-searchbutton"
                 ).click()
                
                sleep(3)

                # 找搜尋結果的div
                focus = driver.find_elements(
                    By.CSS_SELECTOR,
                    "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
                    )[1]

                # 重新定義需要的變數
                offset = 0
                innerHeight = 0
                count = 0  # 累計無效滾動次數
                
            sleep(2)
            
    #尋找所有元素的href    
    href_ = driver.find_elements(
        By.CSS_SELECTOR,
        "div[data-js-log-root] a[aria-label]"
    )
   
    # 放網址的lict
    listData=[]
    counter = 0    
    for href in href_:
        listData.append({
        "href":href.get_attribute("href")
        
    })
        counter+=1
        
    # 把餐廳網址存成json檔        
    with open(f"{folderPath}/WanhuaDistrict.json","a",encoding="utf-8") as f:
        f.write(json.dumps(listData, ensure_ascii=False))
    
    # 清除搜尋的關鍵字
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).clear()   
        
    end = time.time()
    print(f'{link},總共有:{counter}筆,已完成\n')
    print("process_time 測量時間：%f 秒\n" % (end - start))
        
    driver.quit()


In [3]:
def multiprocess():
    
    links = ["萬華區三水街餐廳","萬華區中華路１段餐廳","萬華區中華路２段餐廳","萬華區內江街餐廳","萬華區南寧路餐廳","萬華區和平西路２段餐廳","萬華區和平西路３段餐廳","萬華區國興路餐廳","萬華區大埔街餐廳","萬華區大理街餐廳","萬華區富民路餐廳","萬華區寶興街餐廳","萬華區峨眉街餐廳","萬華區康定路餐廳","萬華區廣州街餐廳","萬華區德昌街餐廳","萬華區忠孝西路２段餐廳","萬華區成都路餐廳","萬華區昆明街餐廳","萬華區東園街餐廳","萬華區柳州街餐廳","萬華區桂林路餐廳","萬華區梧州街餐廳","萬華區武成街餐廳","萬華區武昌街２段餐廳","萬華區民和街餐廳","萬華區水源路餐廳","萬華區永福街餐廳","萬華區汀州路１段餐廳","萬華區洛陽街餐廳","萬華區漢中街餐廳","萬華區漢口街２段餐廳","萬華區環河南路１段餐廳","萬華區環河南路２段餐廳","萬華區環河南路３段餐廳","萬華區興寧街餐廳","萬華區興義街餐廳","萬華區艋舺大道餐廳","萬華區莒光路餐廳","萬華區華西街餐廳","萬華區萬大路餐廳","萬華區萬青街餐廳","萬華區西園路１段餐廳","萬華區西園路２段餐廳","萬華區西寧南路餐廳","萬華區西昌街餐廳","萬華區西藏路餐廳","萬華區貴陽街２段餐廳","萬華區長沙街２段餐廳","萬華區長泰街餐廳","萬華區長順街餐廳","萬華區開封街２段餐廳","萬華區隆昌街餐廳","萬華區雅江街餐廳","萬華區雙和街餐廳","萬華區雙園街餐廳","萬華區青年路餐廳"]
    
    # 平行運算x2,
    with tpe(max_workers=2) as executor:
        executor.map(page_link,links) # 執行page_link涵式,links涵式需要的變數

In [4]:
if __name__=="__main__":
    multiprocess()
    print("抓取全區餐廳的網址已完成")

萬華區中華路１段餐廳需重新整理,搜尋逾時,本次刷新第1次
萬華區三水街餐廳,總共有:120筆,已完成

process_time 測量時間：63.352837 秒

萬華區中華路１段餐廳,總共有:120筆,已完成

process_time 測量時間：122.430957 秒

萬華區中華路２段餐廳,總共有:120筆,已完成

process_time 測量時間：65.727904 秒

萬華區內江街餐廳,總共有:120筆,已完成

process_time 測量時間：63.328696 秒

萬華區南寧路餐廳,總共有:120筆,已完成

process_time 測量時間：63.448017 秒

萬華區和平西路２段餐廳需重新整理,搜尋逾時,本次刷新第1次
萬華區和平西路３段餐廳需重新整理,搜尋逾時,本次刷新第1次
萬華區和平西路２段餐廳,總共有:120筆,已完成

process_time 測量時間：121.870498 秒

萬華區國興路餐廳,總共有:120筆,已完成

process_time 測量時間：63.984249 秒

萬華區大埔街餐廳,總共有:120筆,已完成

process_time 測量時間：66.314597 秒

萬華區大理街餐廳,總共有:120筆,已完成

process_time 測量時間：63.637768 秒

萬華區富民路餐廳,總共有:120筆,已完成

process_time 測量時間：63.488976 秒

萬華區寶興街餐廳,總共有:120筆,已完成

process_time 測量時間：66.520663 秒

萬華區峨眉街餐廳,總共有:120筆,已完成

process_time 測量時間：66.170143 秒

萬華區康定路餐廳,總共有:120筆,已完成

process_time 測量時間：63.758246 秒

萬華區廣州街餐廳,總共有:120筆,已完成

process_time 測量時間：63.293357 秒

萬華區德昌街餐廳,總共有:120筆,已完成

process_time 測量時間：63.310175 秒

萬華區成都路餐廳,總共有:120筆,已完成

process_time 測量時間：65.529501 秒

萬華區昆明街餐廳,總共有:120筆,已完成

process_time 測量

In [ ]:
# folderPath = "WanhuaRestaurant"
# with open(f"{folderPath}/WanhuaRestaurant.json", "r", encoding='utf-8') as file:
    
#     # 取得 json 字串
#     strJson = file.read()
    
#     # 用,分割
#     aList = strJson.split(",")
    

# a = len(aList)
# print(a)
# b = len(set(aList))
# print(b)